# L2: How To Use Structured Outputs

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key

def print_mention(processed_mention, mention):
    # Check if we need to respond
    if processed_mention.needs_response:
        # We need to respond
        print(f"Responding to {processed_mention.sentiment} {processed_mention.product} feedback")
        print(f"  User: {mention}")
        print(f"  Response: {processed_mention.response}")
    else:
        print(f"Not responding to {processed_mention.sentiment} {processed_mention.product} post")
        print(f"  User: {mention}")

    if processed_mention.support_ticket_description:
        print(f"  Adding support ticket: {processed_mention.support_ticket_description}")


KEY = get_openai_api_key()

In [4]:
from openai import OpenAI

# Instantiate the client
client = OpenAI(
    api_key=KEY
)

## Define structure with Pydantic

In [5]:
# The user class from the slides
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    email: Optional[str] = None

In [6]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Make up a user."},
    ],
    response_format=User,
)

In [7]:
user = completion.choices[0].message.parsed
user

User(name='Alice Johnson', age=28, email='alice.johnson@example.com')

## The social media mention structure

In [8]:
from pydantic import BaseModel
from enum import Enum
from typing import List, Optional, Literal
from openai import OpenAI

class Mention(BaseModel):
    # The model chooses the product the mention is about,
    # as well as the social media post's sentiment
    product: Literal['app', 'website', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']

    # Model can choose to respond to the user
    needs_response: bool
    response: Optional[str]

    # If a support ticket needs to be opened,
    # the model can write a description for the
    # developers
    support_ticket_description: Optional[str]

In [9]:
# Example mentions
mentions = [
    # About the app
    "@techcorp your app is amazing! The new design is perfect",
    # Website is down, negative sentiment + needs a fix
    "@techcorp website is down again, please fix!",
    # Nothing to respond to
    "hey @techcorp you're so evil"
]

In [10]:
def analyze_mention(
    mention: str,
    personality: str = "friendly"
) -> Mention:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"""
                Extract structured information from
                social media mentions about our products.

                Provide
                - The product mentioned (website, app, not applicable)
                - The mention sentiment (positive, negative, neutral)
                - Whether to respond (true/false). Don't respond to
                  inflammatory messages or bait.
                - A customized response to send to the user if we need
                  to respond.
                - An optional support ticket description to create.

                Your personality is {personality}.
            """},
            {"role": "user", "content": mention},
        ],
        response_format=Mention,
    )
    return completion.choices[0].message.parsed

In [11]:
print("User post:", mentions[0])
processed_mention = analyze_mention(mentions[0])
processed_mention

User post: @techcorp your app is amazing! The new design is perfect


Mention(product='app', sentiment='positive', needs_response=True, response="Thank you so much for your kind words! We're thrilled to hear that you love the new design! 😊 If you have any feedback or suggestions, feel free to share them with us!", support_ticket_description=None)

In [12]:
rude_mention = analyze_mention(mentions[0], personality="rude")
rude_mention.response

"Thanks for the compliment, but don't get too used to it. It's just a phase!"

In [13]:
mention_json_string = processed_mention.model_dump_json(indent=2)
print(mention_json_string)

{
  "product": "app",
  "sentiment": "positive",
  "needs_response": true,
  "response": "Thank you so much for your kind words! We're thrilled to hear that you love the new design! 😊 If you have any feedback or suggestions, feel free to share them with us!",
  "support_ticket_description": null
}


## You try!

In [14]:
class UserPost(BaseModel):
    message: str

def make_post(output_class):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": """
                You are a customer of Tech Corp (@techcorp), a company
                that provides an app and a website. Create a small
                microblog-style post to them that sends some kind of
                feedback, positive or negative.
            """},
            {"role": "user", "content": "Please write a post."},
        ],
        response_format=output_class,
    )
    return completion.choices[0].message.parsed

new_post = make_post(UserPost)
new_post

UserPost(message='Just wanted to shout out to Tech Corp for their amazing customer service! I had an issue with the app crashing and their support team was super responsive and helped me resolve it quickly. Loving the app more than ever! Keep up the great work!')

In [15]:
analyze_mention(new_post.message)

Mention(product='app', sentiment='positive', needs_response=True, response="Thank you so much for your kind words! We're thrilled to hear that our support team was able to help you quickly. If you need any more assistance or have feedback, feel free to reach out! We appreciate your support!", support_ticket_description=None)

In [16]:
class UserPostWithExtras(BaseModel):
    user_mood: Literal["awful", "bad", "evil"]
    product: Literal['app', 'website', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']
    internal_monologue: List[str]
    message: str

new_post = make_post(UserPostWithExtras)
new_post

UserPostWithExtras(user_mood='bad', product='app', sentiment='negative', internal_monologue=['This app used to be so smooth!', 'Why is it lagging all of a sudden?', 'I need a reliable tool for my daily tasks.'], message="Hey @techcorp, I've been experiencing some major lag issues with your app lately. It wasn’t like this before—can we get a fix? It's really slowing me down!")

In [ ]:
analyze_mention(new_post.message)

## Programming with our mentions

In [17]:
# Loop through posts that tagged us and store the results in a list
rows = []
for mention in mentions:
    # Call the LLM to get a Mention object we can program with
    processed_mention = analyze_mention(mention)

    # Print out some information
    print_mention(processed_mention, mention)

    # Convert our processed data to a dictionary
    # using Pydantic tools
    processed_dict = processed_mention.model_dump()

    # Store the original message in the dataframe row
    processed_dict['mention'] = mention
    rows.append(processed_dict)

    print("") # Add separator to make it easier to read

Responding to positive app feedback
  User: @techcorp your app is amazing! The new design is perfect
  Response: Thank you so much for your kind words! We're thrilled to hear that you love the new design of our app. If you have any suggestions or feedback, feel free to share!

Responding to negative website feedback
  User: @techcorp website is down again, please fix!
  Response: Hi there! We're sorry to hear you're experiencing issues with our website. We're currently working to resolve it as quickly as possible. Thank you for your patience!
  Adding support ticket: User reported that the website is down and requested a fix.

Not responding to negative not_applicable post
  User: hey @techcorp you're so evil



In [18]:
import pandas as pd

df = pd.DataFrame(rows)
df

,product,sentiment,needs_response,response,support_ticket_description,mention
0,app,positive,True,Thank you so much for your kind words! We're t...,None,@techcorp your app is amazing! The new design ...
1,website,negative,True,Hi there! We're sorry to hear you're experienc...,User reported that the website is down and req...,"@techcorp website is down again, please fix!"
2,not_applicable,negative,False,None,None,hey @techcorp you're so evil
